<div align='left' style="width:29%;overflow:hidden;">
<a href='http://inria.fr'>
<img src='https://github.com/lmarti/jupyter_custom/raw/master/imgs/inr_logo_rouge.png' alt='Inria logo' title='Inria logo'/>
</a>
</div>

# Representations, LDA and topics in CORD-19

> Here we calculate representations of the papers based on their text content. Then, from these representations, a modeling of topics will be carried out using the LDA method. Finally, the most relevant papers for each topic will be determined using the PageRank scores of each paper.

In [ ]:
# default_exp lda

In [1]:
!pip install -q -r requirements.txt

In [2]:
# export
from risotto.references import load_papers_from_metadata_file, build_papers_reference_graph, paper_as_markdown
from fastprogress.fastprogress import progress_bar

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from pathlib import Path

import scispacy
import en_core_sci_sm
import networkx as nx
from collections import defaultdict
import numpy as np

Loading paper dataset and re-generating the graph of papers and the corresponding PageRank.

In [3]:
cord19_dataset_folder = "./datasets/CORD-19-research-challenge"

In [4]:
papers, _ = load_papers_from_metadata_file(cord19_dataset_folder)

In [5]:
G = build_papers_reference_graph(papers)

In [6]:
pageranks = nx.pagerank(G)

## Paper representations

In order to build a representation for each paper, the following libraries will be used:

- spaCy: https://spacy.io/
- scispaCy: https://allenai.github.io/scispacy/

The language model named`en_core_sci_sm` will be used, which has been trained with a corpus of biomedical text with a vocabulary of more than 100.000 words.
In case of needing a model with a larger vocabulary, there are some others available.

Loading the biomedical language pipeline.

In [7]:
# export
nlp = en_core_sci_sm.load()

In [8]:
# Select a paper to showcase spacy's features
sample_paper = list(pageranks.keys())[0]
sample_text = "\n".join([ paragraph["text"] for paragraph in sample_paper._file_contents["body_text"]])
sample_text

doc = nlp(sample_text, disable=["tagger", "parser", "ner"])
doc[17].lemma_

'be'

The document tokenized by the `spacy` pipeline is displayed.
An interesting thing about using `spacy` with the pretrained language model is that it automatically computes document and token representations vectors.
It's a pending task to find out which language model architecture it's used to compute those vectors.

A relevant aspect that influences downstream tasks is the number of out-of-vocabulary (OOV) tokens.
The following cell makes a quick inspection over a sample paper counting the number of OOV tokens.
A continuación, se realizará una iteración sobre los tokens para detectarlos.

In [9]:
num_oov = 0
for token in doc:
    if token.is_oov and token.string != "\n":
        if token.string.endswith("virus"):
            print(token, "not found")
        num_oov += 1
    else:
        if token.string.endswith("virus"):
            print(token, "found")

virus found
virus found
virus found


In [10]:
print(f'Number of out of vocabulary tokens: {num_oov} ({100 * num_oov / len(doc)}%).')

Number of out of vocabulary tokens: 336 (4.801371820520148%).


Note that relevant tokens, such as *coronavirus* are included in the language model vocabulary.

Testing the mechanisms used to remove stopwords, punctuation, spaces, and extract the token's lemma.

In [11]:
tokens = {token for token in doc}

In [12]:
no_stop_word_tokens = {token for token in doc if not (token.is_stop or token.is_punct or token.is_space)}

In [13]:
len(tokens), len(no_stop_word_tokens)

(6998, 3544)

## Latent Dirichlet Allocation (LDA)

The following cells will perform topic modelling experiments using the LDA technique.
The `scikit-learn` implementation of this model will be used.

First, let's process all documents texts.

In [14]:
# export
def process_papers_file_contents(papers):
    texts = []
    nlp = en_core_sci_sm.load()
    for paper in progress_bar(papers):
        text = " \n ".join([ paragraph["text"] for paragraph in paper._file_contents["body_text"]])
        """
        NB.: for development speed purposes, the only document's attributes
        considered for the topic modelling were the title and the abstract.
        Should the text be included in other experiments, the following line
        should be modified to include `{paper.text}`.
        """
        texts.append(f"{paper.title} \n {paper.abstract}")
    return texts

In [15]:
docs = process_papers_file_contents(list(pageranks.keys()))

Peeking at the top 5 papers.

In [16]:
print('\n=====\n'.join(docs[:5]))

Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis 
 Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused to different sequences (‘bodies’) derived from the 3′ end. Their generation involves a unique mechanism of discontinuous subgenomic RNA synthesis that resembles copy-choice RNA recombination. During this process, the nascent RNA strand is transferred from one site in the template to another, during either plus or minus strand synthesis, to yield subgenomic RNA molecules. Central to this process are transcription-regulating sequences (TRSs), which are present at both template sites and ensure the fidelity of strand transfer. Here we present results of a comprehensive co-variation mutagenesis study of equine arteritis virus TRSs, demonstrating that discontinuous RNA synthesis depends not only on base pairing between sense leader TRS and antisense body TRS, but also on the primary sequence o

Vectors storing the token occurrence count will be used as document representations.
`tf-idf` vectors are purposefully not used because the document frequency normalization is already carried out by the LDA technique.

In [17]:
# export
def tokenizer(sentence):
    tokens = []
    for token in nlp(sentence, disable=["tagger", "parser", "ner"]):
        # Se descartan números, stopwords, puntuación, espacio y tokens de largo 1
        if not (token.like_num or token.is_stop or token.is_punct
                or token.is_space or len(token) == 1):
            tokens.append(token.lemma_)
    return tokens

In [18]:
count_vectorizer = CountVectorizer(tokenizer=tokenizer, lowercase=True)

In [19]:
vectorized_docs = count_vectorizer.fit_transform(docs)

A sparse matrix is built rows one for each document, and columns: one for each token.

In [20]:
vectorized_docs.shape

(44648, 164664)

In [21]:
len(count_vectorizer.vocabulary_)

164664

In [22]:
lda = LatentDirichletAllocation(n_components=10, verbose=2, n_jobs=-1)
lda = lda.fit(vectorized_docs)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   46.8s remaining:   46.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   49.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 1 of max_iter: 10


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   32.2s remaining:   32.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   34.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 2 of max_iter: 10


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   27.9s remaining:   27.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   30.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/Users/lmarti/.pyenv/versions/3.8.2/envs/risotto/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  1.2min remaining:  1.2min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed: 24.6min finished


iteration: 9 of max_iter: 10


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   47.0s remaining:   47.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   49.3s finished


iteration: 10 of max_iter: 10


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   54.8s remaining:   54.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.0min finished


In [23]:
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=2)

The execution of the following cells will display the most relevant tokens for each identified topic.

In [24]:
# export
def print_topic_words(topic_model, vectorizer, num_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(topic_model.components_):
        message = f"Topic {topic_idx}: " 
        message += " ".join(
            [feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]])
        print(message)

In [25]:
print_topic_words(lda, count_vectorizer, 20)

Topic 0: mers-cov vaccine virus coronavirus mouse middle east antibody strain syndrome high group ibv tgev effect show day study respiratory infection
Topic 1: virus sample respiratory sequence detection detect assay strain test coronavirus study infection analysis human pcr result method isolate positive gene
Topic 2: calve study diarrhea pedv pig group result high infection effect control day animal porcine air increase piglet sample herd abstract
Topic 3: health disease outbreak case model datum public epidemic covid-19 transmission pandemic infectious risk control study country china system result spread
Topic 4: patient respiratory infection clinical case pneumonia study covid-19 disease acute severe nan influenza hospital treatment de child age care symptom
Topic 5: protein bind activity structure de domain peptide membrane fusion acid protease receptor coronavirus sars-cov inhibitor et enzyme ace2 site residue
Topic 6: cat feline model influenza fip prediction result rate method

The dataset papers will be classified into the different previously modelled topics.

In [26]:
docs_classified = lda.transform(vectorized_docs)
docs_classified[:5]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed: 47.5min remaining: 47.5min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed: 47.6min finished


array([[8.19751514e-04, 8.19851715e-04, 8.19785769e-04, 8.19772231e-04,
        8.19739964e-04, 8.19814598e-04, 8.19997258e-04, 9.92621681e-01,
        8.19767690e-04, 8.19838685e-04],
       [1.25023508e-03, 1.25015511e-03, 1.25002050e-03, 1.25010727e-03,
        1.25004174e-03, 8.04528155e-01, 1.25003536e-03, 1.85470640e-01,
        1.25031194e-03, 1.25029797e-03],
       [1.09919490e-03, 1.09924208e-03, 1.09941836e-03, 1.09920295e-03,
        4.52019223e-02, 7.13667213e-02, 1.09896932e-03, 7.16309239e-01,
        3.03368599e-02, 1.31289229e-01],
       [8.93201291e-04, 8.93060812e-04, 8.92943176e-04, 8.93036352e-04,
        8.92952591e-04, 8.69791018e-01, 3.50719363e-02, 8.88857918e-02,
        8.92976462e-04, 8.93082942e-04],
       [8.40432330e-04, 4.05902690e-02, 8.40452815e-04, 8.40620333e-04,
        8.40494481e-04, 8.40471730e-04, 8.40428417e-04, 9.52685855e-01,
        8.40433000e-04, 8.40542427e-04]])

Finalmente, para cada uno de los temas identifiados, se imprimen los top-5 papers pertenecientes al tema, ordenados por su *pagerank*.
Finally, the top-5 PageRank sorted papers belonging to each topic are displayed.

In [27]:
docs_topics = docs_classified.argmax(1)
topic_papers = defaultdict(list)
all_papers = list(pageranks.keys())
for idx, topic_id in enumerate(docs_topics):
    topic_papers[topic_id].append(all_papers[idx])
    
for topic_id, papers in sorted(topic_papers.items(), key=lambda t: t[0]):
    print(f"Topic ID {topic_id}")
    sorted_papers = sorted(papers, reverse=True, key=lambda p: pageranks[p])
    for paper in sorted_papers[:5]:
        paper_as_markdown(paper)
    #print("\n", end="")

Topic ID 0



- **Title:** Detection of hepatitis C virus in the nasal secretions of an intranasal drug-user
- **Authors:** McMahon, James M; Simm, Malgorzata; Milano, Danielle; Clatts, Michael
- **Publish date/time:** 2004-05-07
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: One controversial source of infection for hepatitis C virus (HCV) involves the sharing of contaminated implements, such as straws or spoons, used to nasally inhale cocaine and other powdered drugs. An essential precondition for this mode of transmission is the presence of HCV in the nasal secretions of intranasal drug users. METHODS: Blood and nasal secretion samples were collected from five plasma-positive chronic intranasal drug users and tested for HCV RNA using RT-PCR. RESULTS: HCV was detected in all five blood samples and in the nasal secretions of the subject with the highest serum viral load. CONCLUSIONS: This study is the first to demonstrate the presence of HCV in nasal secretions. This finding has implications for potential transmission of HCV through contact with contaminated nasal secretions.


- **Title:** The influence of locked nucleic acid residues on the thermodynamic properties of 2′-O-methyl RNA/RNA heteroduplexes
- **Authors:** Kierzek, Elzbieta; Ciesielska, Anna; Pasternak, Karol; Mathews, David H.; Turner, Douglas H.; Kierzek, Ryszard
- **Publish date/time:** 2005-09-09
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** The influence of locked nucleic acid (LNA) residues on the thermodynamic properties of 2′-O-methyl RNA/RNA heteroduplexes is reported. Optical melting studies indicate that LNA incorporated into an otherwise 2′-O-methyl RNA oligonucleotide usually, but not always, enhances the stabilities of complementary duplexes formed with RNA. Several trends are apparent, including: (i) a 3′ terminal U LNA and 5′ terminal LNAs are less stabilizing than interior and other 3′ terminal LNAs; (ii) most of the stability enhancement is achieved when LNA nucleotides are separated by at least one 2′-O-methyl nucleotide; and (iii) the effects of LNA substitutions are approximately additive when the LNA nucleotides are separated by at least one 2′-O-methyl nucleotide. An equation is proposed to approximate the stabilities of complementary duplexes formed with RNA when at least one 2′-O-methyl nucleotide separates LNA nucleotides. The sequence dependence of 2′-O-methyl RNA/RNA duplexes appears to be similar to that of RNA/RNA duplexes, and preliminary nearest-neighbor free energy increments at 37°C are presented for 2′-O-methyl RNA/RNA duplexes. Internal mismatches with LNA nucleotides significantly destabilize duplexes with RNA.


- **Title:** Ethnoveterinary medicines used for ruminants in British Columbia, Canada
- **Authors:** Lans, Cheryl; Turner, Nancy; Khan, Tonya; Brauer, Gerhard; Boepple, Willi
- **Publish date/time:** 2007-02-26
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: The use of medicinal plants is an option for livestock farmers who are not allowed to use allopathic drugs under certified organic programs or cannot afford to use allopathic drugs for minor health problems of livestock. METHODS: In 2003 we conducted semi-structured interviews with 60 participants obtained using a purposive sample. Medicinal plants are used to treat a range of conditions. A draft manual prepared from the data was then evaluated by participants at a participatory workshop. RESULTS: There are 128 plants used for ruminant health and diets, representing several plant families. The following plants are used for abscesses: Berberis aquifolium/Mahonia aquifolium Echinacea purpurea, Symphytum officinale, Bovista pila, Bovista plumbea, Achillea millefolium and Usnea longissima. Curcuma longa L., Salix scouleriana and Salix lucida are used for caprine arthritis and caprine arthritis encephalitis.Euphrasia officinalis and Matricaria chamomilla are used for eye problems. Wounds and injuries are treated with Bovista spp., Usnea longissima, Calendula officinalis, Arnica sp., Malva sp., Prunella vulgaris, Echinacea purpurea, Berberis aquifolium/Mahonia aquifolium, Achillea millefolium, Capsella bursa-pastoris, Hypericum perforatum, Lavandula officinalis, Symphytum officinale and Curcuma longa. Syzygium aromaticum and Pseudotsuga menziesii are used for coccidiosis. The following plants are used for diarrhea and scours: Plantago major, Calendula officinalis, Urtica dioica, Symphytum officinale, Pinus ponderosa, Potentilla pacifica, Althaea officinalis, Anethum graveolens, Salix alba and Ulmus fulva. Mastitis is treated with Achillea millefolium, Arctium lappa, Salix alba, Teucrium scorodonia and Galium aparine. Anethum graveolens and Rubus sp., are given for increased milk production.Taraxacum officinale, Zea mays, and Symphytum officinale are used for udder edema. Ketosis is treated with Gaultheria shallon, Vaccinium sp., and Symphytum officinale. Hedera helix and Alchemilla vulgaris are fed for retained placenta. CONCLUSION: Some of the plants showing high levels of validity were Hedera helix for retained placenta and Euphrasia officinalis for eye problems. Plants with high validity for wounds and injuries included Hypericum perforatum, Malva parviflora and Prunella vulgaris. Treatments with high validity against endoparasites included those with Juniperus communis and Pinus ponderosa. Anxiety and pain are well treated with Melissa officinalis and Nepeta caesarea.


- **Title:** Arthritis suppression by NADPH activation operates through an interferon-β pathway
- **Authors:** Olofsson, Peter; Nerstedt, Annika; Hultqvist, Malin; Nilsson, Elisabeth C; Andersson, Sofia; Bergelin, Anna; Holmdahl, Rikard
- **Publish date/time:** 2007-05-09
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: A polymorphism in the activating component of the nicotinamide adenine dinucleotide phosphate (NADPH) oxidase complex, neutrophil cytosolic factor 1 (NCF1), has previously been identified as a regulator of arthritis severity in mice and rats. This discovery resulted in a search for NADPH oxidase-activating substances as a potential new approach to treat autoimmune disorders such as rheumatoid arthritis (RA). We have recently shown that compounds inducing NCF1-dependent oxidative burst, e.g. phytol, have a strong ameliorating effect on arthritis in rats. However, the underlying molecular mechanism is still not clearly understood. The aim of this study was to use gene-expression profiling to understand the protective effect against arthritis of activation of NADPH oxidase in the immune system. RESULTS: Subcutaneous administration of phytol leads to an accumulation of the compound in the inguinal lymph nodes, with peak levels being reached approximately 10 days after administration. Hence, global gene-expression profiling on inguinal lymph nodes was performed 10 days after the induction of pristane-induced arthritis (PIA) and phytol administration. The differentially expressed genes could be divided into two pathways, consisting of genes regulated by different interferons. IFN-γ regulated the pathway associated with arthritis development, whereas IFN-β regulated the pathway associated with disease protection through phytol. Importantly, these two molecular pathways were also confirmed to differentiate between the arthritis-susceptible dark agouti (DA) rat, (with an Ncf-1(DA )allele that allows only low oxidative burst), and the arthritis-protected DA.Ncf-1(E3 )rat (with an Ncf1(E3 )allele that allows a stronger oxidative burst). CONCLUSION: Naturally occurring genetic polymorphisms in the Ncf-1 gene modulate the activity of the NADPH oxidase complex, which strongly regulates the severity of arthritis. We now show that the Ncf-1 allele that enhances oxidative burst and protects against arthritis is operating through an IFN-β-associated pathway, whereas the arthritis-driving allele operates through an IFN-γ-associated pathway. Treatment of arthritis-susceptible rats with an NADPH oxidase-activating substance, phytol, protects against arthritis. Interestingly, the treatment led to a restoration of the oxidative-burst effect and induction of a strikingly similar IFN-β-dependent pathway, as seen with the disease-protective Ncf1 polymorphism.

...


- **Title:** Discovering human history from stomach bacteria
- **Authors:** Disotell, Todd R
- **Publish date/time:** 2003-04-28
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Recent analyses of human pathogens have revealed that their evolutionary histories are congruent with the hypothesized pattern of ancient and modern human population migrations. Phylogenetic trees of strains of the bacterium Helicobacter pylori and the polyoma JC virus taken from geographically diverse groups of human beings correlate closely with relationships of the populations in which they are found.


- **Title:** Viral Discovery and Sequence Recovery Using DNA Microarrays
- **Authors:** Wang, David; Urisman, Anatoly; Liu, Yu-Tsueng; Springer, Michael; Ksiazek, Thomas G; Erdman, Dean D; Mardis, Elaine R; Hickenbotham, Matthew; Magrini, Vincent; Eldred, James; Latreille, J. Phillipe; Wilson, Richard K; Ganem, Don; DeRisi, Joseph L
- **Publish date/time:** 2003-11-17
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Because of the constant threat posed by emerging infectious diseases and the limitations of existing approaches used to identify new pathogens, there is a great demand for new technological methods for viral discovery. We describe herein a DNA microarray-based platform for novel virus identification and characterization. Central to this approach was a DNA microarray designed to detect a wide range of known viruses as well as novel members of existing viral families; this microarray contained the most highly conserved 70mer sequences from every fully sequenced reference viral genome in GenBank. During an outbreak of severe acute respiratory syndrome (SARS) in March 2003, hybridization to this microarray revealed the presence of a previously uncharacterized coronavirus in a viral isolate cultivated from a SARS patient. To further characterize this new virus, approximately 1 kb of the unknown virus genome was cloned by physically recovering viral sequences hybridized to individual array elements. Sequencing of these fragments confirmed that the virus was indeed a new member of the coronavirus family. This combination of array hybridization followed by direct viral sequence recovery should prove to be a general strategy for the rapid identification and characterization of novel viruses and emerging infectious disease.


- **Title:** Virology on the Internet: the time is right for a new journal
- **Authors:** Garry, Robert F
- **Publish date/time:** 2004-08-26
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Virology Journal is an exclusively on-line, Open Access journal devoted to the presentation of high-quality original research concerning human, animal, plant, insect bacterial, and fungal viruses. Virology Journal will establish a strategic alternative to the traditional virology communication process.


- **Title:** Locked nucleic acid (LNA) mediated improvements in siRNA stability and functionality
- **Authors:** Elmén, Joacim; Thonberg, Håkan; Ljungberg, Karl; Frieden, Miriam; Westergaard, Majken; Xu, Yunhe; Wahren, Britta; Liang, Zicai; Ørum, Henrik; Koch, Troels; Wahlestedt, Claes
- **Publish date/time:** 2005-01-14
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Therapeutic application of the recently discovered small interfering RNA (siRNA) gene silencing phenomenon will be dependent on improvements in molecule bio-stability, specificity and delivery. To address these issues, we have systematically modified siRNA with the synthetic RNA-like high affinity nucleotide analogue, Locked Nucleic Acid (LNA). Here, we show that incorporation of LNA substantially enhances serum half-life of siRNA's, which is a key requirement for therapeutic use. Moreover, we provide evidence that LNA is compatible with the intracellular siRNA machinery and can be used to reduce undesired, sequence-related off-target effects. LNA-modified siRNAs targeting the emerging disease SARS, show improved efficiency over unmodified siRNA on certain RNA motifs. The results from this study emphasize LNA's promise in converting siRNA from a functional genomics technology to a therapeutic platform.


- **Title:** Detection and characterization of horizontal transfers in prokaryotes using genomic signature
- **Authors:** Dufraigne, Christine; Fertil, Bernard; Lespinats, Sylvain; Giron, Alain; Deschavanne, Patrick
- **Publish date/time:** 2005-01-13
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Horizontal DNA transfer is an important factor of evolution and participates in biological diversity. Unfortunately, the location and length of horizontal transfers (HTs) are known for very few species. The usage of short oligonucleotides in a sequence (the so-called genomic signature) has been shown to be species-specific even in DNA fragments as short as 1 kb. The genomic signature is therefore proposed as a tool to detect HTs. Since DNA transfers originate from species with a signature different from those of the recipient species, the analysis of local variations of signature along recipient genome may allow for detecting exogenous DNA. The strategy consists in (i) scanning the genome with a sliding window, and calculating the corresponding local signature (ii) evaluating its deviation from the signature of the whole genome and (iii) looking for similar signatures in a database of genomic signatures. A total of 22 prokaryote genomes are analyzed in this way. It has been observed that atypical regions make up ∼6% of each genome on the average. Most of the claimed HTs as well as new ones are detected. The origin of putative DNA transfers is looked for among ∼12 000 species. Donor species are proposed and sometimes strongly suggested, considering similarity of signatures. Among the species studied, Bacillus subtilis, Haemophilus Influenzae and Escherichia coli are investigated by many authors and give the opportunity to perform a thorough comparison of most of the bioinformatics methods used to detect HTs.

---

In [1]:
# tell nbdev to generate library from notebooks
from nbdev.export import *
notebook2script()

Converted 00_downloader.ipynb.
Converted 01_references.ipynb.
Converted 02_representations_and_lda.ipynb.
Converted 03_hierarchical_topic_modelling.ipynb.
Converted 99_risotto_gui.ipynb.
Converted index.ipynb.


In [5]:
# this code is here for cosmetic reasons
from IPython.core.display import HTML
from urllib.request import urlopen
HTML(urlopen('https://raw.githubusercontent.com/lmarti/jupyter_custom/master/custom.include').read().decode('utf-8'))

---